# Build convrnn

So we've downloaded the dataset and analyzed and visualized it. We're ready now for the next step: building the convrnn.

We first test it with a synthetic dataset: a ball that bounces.

In [16]:
%load_ext autoreload

%autoreload

from synthetic_datasets import Ball
import utils

# Give init parameters here
b = Ball()
# Calling an instance of Ball generates a batch of images
utils.plotting.plot_synthetic(b(batch_size=10))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
